In [1]:
!pip -q install bandit semgrep pip-audit lief

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.3/89.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.5/134.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.6/231.6 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 753.1/753.1 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.7/193.7 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.0/375.0 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.4/120.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
import os, textwrap, subprocess, sys, json, pathlib

WORK = "/kaggle/working"
TARGET_DIR = os.path.join(WORK, "target_project")
os.makedirs(TARGET_DIR, exist_ok=True)

In [3]:
# --- Create a tiny demo project to scan (replace with your own path later) ---
demo_py = r"""
import os, pickle, subprocess

def bad1(user_input):
    # command injection risk
    os.system("ls " + user_input)

def bad2(s):
    # code injection risk
    return eval(s)

def bad3(blob):
    # unsafe deserialization risk
    return pickle.loads(blob)

def bad4(path):
    # risky shell usage
    subprocess.run("cat " + path, shell=True)
"""

with open(os.path.join(TARGET_DIR, "app.py"), "w") as f:
    f.write(demo_py)

# optional: requirements to audit (edit to your real requirements.txt)
with open(os.path.join(TARGET_DIR, "requirements.txt"), "w") as f:
    f.write("requests==2.19.0\n")  # intentionally old example

In [4]:
print("=== 1) Bandit (Python SAST) ===")
subprocess.run(["bandit", "-q", "-r", TARGET_DIR, "-f", "txt"])

=== 1) Bandit (Python SAST) ===
Run started:2026-02-16 22:27:22.295803+00:00

Test results:
>> Issue: [B403:blacklist] Consider possible security implications associated with pickle module.
   Severity: Low   Confidence: High
   CWE: CWE-502 (https://cwe.mitre.org/data/definitions/502.html)
   More Info: https://bandit.readthedocs.io/en/1.9.3/blacklists/blacklist_imports.html#b403-import-pickle
   Location: /kaggle/working/target_project/app.py:2:0
1	
2	import os, pickle, subprocess
3	

--------------------------------------------------
>> Issue: [B605:start_process_with_a_shell] Starting a process with a shell, possible injection detected, security issue.
   Severity: High   Confidence: High
   CWE: CWE-78 (https://cwe.mitre.org/data/definitions/78.html)
   More Info: https://bandit.readthedocs.io/en/1.9.3/plugins/b605_start_process_with_a_shell.html
   Location: /kaggle/working/target_project/app.py:6:4
5	    # command injection risk
6	    os.system("ls " + user_input)
7	

----------

CompletedProcess(args=['bandit', '-q', '-r', '/kaggle/working/target_project', '-f', 'txt'], returncode=1)

In [5]:
print("\n=== 2) Semgrep (rules-based SAST) ===")
# auto rules are broad; you can also use specific rulesets like p/python, etc.
subprocess.run(["semgrep", "--quiet", "--config", "p/python", TARGET_DIR])


=== 2) Semgrep (rules-based SAST) ===
                  
                  
┌────────────────┐
│ 1 Code Finding │
└────────────────┘
                                         
    /kaggle/working/target_project/app.py
   ❯❯❱ python.lang.security.audit.subprocess-shell-true.subprocess-shell-true
          Found 'subprocess' function 'run' with 'shell=True'. This is dangerous because this call will spawn
          the command using a shell process. Doing so propagates current shell settings and variables, which 
          makes it much easier for a malicious actor to execute commands. Use 'shell=False' instead.         
          Details: https://sg.run/J92w                                                                       
                                                                                                             
           ▶▶┆ Autofix ▶ False
           18┆ subprocess.run("cat " + path, shell=True)



CompletedProcess(args=['semgrep', '--quiet', '--config', 'p/python', '/kaggle/working/target_project'], returncode=0)

In [6]:
print("\n=== 3) pip-audit (dependency vulnerabilities) ===")
# audits requirements file (known CVEs)
subprocess.run(["pip-audit", "-r", os.path.join(TARGET_DIR, "requirements.txt")])


=== 3) pip-audit (dependency vulnerabilities) ===


Traceback (most recent call last):
  File "/usr/local/bin/pip-audit", line 8, in <module>
    sys.exit(audit())
             ^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip_audit/_cli.py", line 538, in audit
    for spec, vulns in auditor.audit(source):
                       ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip_audit/_audit.py", line 68, in audit
    for dep, vulns in self._service.query_all(specs):
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip_audit/_service/interface.py", line 155, in query_all
    for spec in specs:
                ^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip_audit/_dependency_source/requirement.py", line 134, in collect
    yield from self._collect_from_files(collect_files)
  File "/usr/local/lib/python3.12/dist-packages/pip_audit/_dependency_source/requirement.py", line 180, in _collect_from_files
    ve.create(ve_dir)
  File "/usr/local/lib/py

CompletedProcess(args=['pip-audit', '-r', '/kaggle/working/target_project/requirements.txt'], returncode=1)

In [7]:
print("\n=== 4) (Optional) ELF hardening check example with LIEF ===")
# If you have an ELF file to analyze, set ELF_PATH to it.
# Example: ELF_PATH = "/kaggle/input/your-dataset/some_binary"
ELF_PATH = None

if ELF_PATH and os.path.exists(ELF_PATH):
    import lief
    bin = lief.parse(ELF_PATH)
    print("File:", ELF_PATH)
    # Heuristics for common mitigations (Linux ELF)
    is_pie = getattr(bin, "is_pie", False)
    has_nx = getattr(bin, "has_nx", False)
    has_canary = getattr(bin, "has_symbol", lambda x: False)("__stack_chk_fail")
    relro = "unknown"
    try:
        relro = "full" if bin.has_full_relro else ("partial" if bin.has_relro else "none")
    except Exception:
        pass

    print("PIE:", bool(is_pie))
    print("NX :", bool(has_nx))
    print("Canary:", bool(has_canary))
    print("RELRO:", relro)

    # Dangerous C library calls (very rough): check imported symbols
    risky = {"gets", "strcpy", "strcat", "sprintf", "vsprintf"}
    imported = set()
    try:
        for s in bin.imported_symbols:
            imported.add(s.name)
    except Exception:
        pass
    found = sorted(risky.intersection(imported))
    print("Risky imports found:", found if found else "none detected")
else:
    print("ELF_PATH not set. If you upload a binary, set ELF_PATH to analyze hardening flags.")


=== 4) (Optional) ELF hardening check example with LIEF ===
ELF_PATH not set. If you upload a binary, set ELF_PATH to analyze hardening flags.
